In [ ]:
# Imports
import pandas as pd
import os
import matplotlib.pyplot as plt
from pypmml import Model
from ReadOPCUA import read_opcua_values
import numpy as np
from pypmml import Model
from sklearn_pandas import DataFrameMapper
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.gridspec as gridspec

### Read current paramters, load models and visualize optimization potential (Colored version for application)

**Online mode (PLC running):** Automatic readout via PLC of current parameters. The data point adresses are defined in the knowledge base and can be modified [there.](./KnowledgeBase.ipynb)

**Offline mode (PLC not running):** Manual entry of current parameter settings.

In [ ]:
%run KnowledgeBase.ipynb
%store -r server_url
%store -r node_ids

online_mode = False  # Set to True for online mode, False for offline mode


def get_value_within_range(prompt, min_value, max_value):
    """
    Function for manual entry of current parameter settings. 
    The min_value and max_value are technological boundary values defined in the knowledge base.

    Parameters:
    - promt (string): message 
    - min_value (float): Minimum value of paramter range.
    - max_value (float): Maximum value of paramter range.
    """
    
    while True:
        try:
            value = float(input(prompt))
            if min_value <= value <= max_value:
                return value
            else:
                print("\033[1;31mWarning: Value must be between {} and {}.\033[0m".format(min_value, max_value))
        except ValueError:
            print("\033[1;31mInvalid input. Please enter a number.\033[0m")

# Online mode
if online_mode:
    # Online mode
    try:
        
        # Creates a list of the node IDS extracted from the node_ids_dict from the knowledge base
        node_ids_list = list(node_ids_dict.values())
        
        # Get current parameters for the given node_ids_list via OPC UA
        result = read_opcua_values(server_url, node_ids_list)
        
        #  Dictionary, where the keys are the variable names (e.g. T_cleaning) from node_ids_dict, and the values are the current paramters
        result_dict = {key: value for key, value in zip(node_ids_dict.keys(), [x[1] for x in result])}
        
        # Extract individual variables
        T_fluid = result_dict['T_cleaning']
        p_cleaning = result_dict['p_cleaning']
        p_rinsing = result_dict['p_rinsing']
        T_drying = result_dict['T_drying']
        n_drying = result_dict['n_drying']
    
        print("T_fluid:", T_cleaning)
        print("p_cleaning:", p_cleaning)
        print("p_rinsing:", p_rinsing)
        print("T_drying:", T_drying)
        print("n_drying:", n_drying)
        
    except Exception as e:
        print("Error occurred while fetching data:", e)
        online_mode = False  # Switch to offline mode if an error occurs
        
# Offline mode  
if not online_mode:
        
    # Offline mode  
    try:
        n_drying = get_value_within_range("Enter current value for n_drying in rpm: ", 860, 3600)
        p_cleaning = get_value_within_range("Enter current value for p_cleaning in bar: ", 0.05, 2.3)
        p_rinsing = get_value_within_range("Enter current value for p_rinsing in bar: ", 0.05, 2.3)
        T_drying = get_value_within_range("Enter current value for T_drying in °C: ", 45, 115)
        T_fluid = get_value_within_range("Enter current value for T_fluid in °C: ", 40, 70)

    except Exception as e:
        print("Error occurred in offline mode:", e)
        

# Load the PMML file
current_path = os.getcwd()
model_drying_fan_speed_file_path = os.path.join(current_path +'/polynomial_regression_models/model_drying_fan_speed.pmml')
model_cleaning_pump_pressure_file_path = os.path.join(current_path +'/polynomial_regression_models/model_cleaning_pump_pressure.pmml')
model_rinsing_pump_pressure_file_path = os.path.join(current_path +'/polynomial_regression_models/model_rinsing_pump_pressure.pmml')
model_fluid_temperature_file_path = os.path.join(current_path +'/polynomial_regression_models/model_fluid_temperature.pmml')
model_drying_air_temperature_file_path = os.path.join(current_path +'/polynomial_regression_models/model_drying_air_temperature.pmml')


model_drying_fan_speed = Model.fromFile(model_drying_fan_speed_file_path)
model_cleaning_pump_pressure = Model.fromFile(model_cleaning_pump_pressure_file_path)
model_rinsing_pump_pressure = Model.fromFile(model_rinsing_pump_pressure_file_path)
model_fluid_temperature =  Model.fromFile(model_fluid_temperature_file_path)
model_drying_air_temperature = Model.fromFile(model_drying_air_temperature_file_path)

# Visulization of optimization potential

def plot_and_fill_area(ax, min_value, max_value, current_value, optimum_value, regression_model, plot_title, x_label):
    """
    Plot the regression model with the predicted value and fill the area
    between the operating point and the optimal operating point.

    Parameters:
    - ax: Axes object to plot on.
    - min_value (float): Minimum value of paramter range.
    - max_value (float): Maximum value of paramter range.
    - current_value (float): The current operating point.
    - optimum_value (float): The optimal operating point.
    - regression_model: The trained PMML model for predicting the active power.
    - plot_title(string): Title for plot.
    - x_label (string): Label for x-axis.
    """
    # Use the PMML model for prediction
    P = regression_model.predict([current_value])
    P_opt = regression_model.predict([optimum_value])

    # Plot the regression model with the predicted value
    x_values = np.linspace(min_value, max_value, 100).reshape(-1, 1)
    y_values = regression_model.predict(x_values)

    ax.plot(x_values, y_values, color="black", label='Regression Model')
    ax.scatter(current_value, P[0], color="red", alpha=0.5, label="Operating point")
    ax.scatter(optimum_value, P_opt[0], color="green", alpha=0.5, label="Active power optimum operating point")

    # Fill area under the curve between operating point and optimal operating point
    y_values = [item for sublist in y_values for item in sublist]
    ax.fill_between(x_values.ravel(), y_values, where=(x_values.ravel() >= optimum_value) & (x_values.ravel() <= current_value),  color='green', alpha=0.3, label='Energy optimization potential')

    ax.set_xlabel(x_label)
    ax.set_ylabel(r'$\overline{P}$ in W')
    ax.legend()
    ax.set_title(plot_title)
    ax.grid(True)

    return P, P_opt


# Create figure and specify grid dimensions
fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(2, 3, figure=fig)

# Specify subplot locations
ax1 = fig.add_subplot(gs[0, 0])  # First row, first column
ax2 = fig.add_subplot(gs[0, 1])  # First row, second column
ax3 = fig.add_subplot(gs[0, 2])  # First row, third column
ax4 = fig.add_subplot(gs[1, 0])  # Second row, span first and second columns
ax5 = fig.add_subplot(gs[1, 1])  # Second row, third column

# Call the function with your data for each subplot
plot_and_fill_area(ax5, 860, 3600, n_drying, 860, model_drying_fan_speed, 'Drying fan speed', r'$\mathit{n}_{\mathrm{drying}}$ in rpm')
plot_and_fill_area(ax2, 0.05, 2.3, p_cleaning, 0.05, model_cleaning_pump_pressure, 'Cleaning pump pressure', r'$\mathit{p}_{\mathrm{cleaning}}$ in bar')
plot_and_fill_area(ax3, 0.05, 2.3, p_rinsing, 0.05, model_rinsing_pump_pressure, 'Rinsing pump pressure', r'$\mathit{p}_{\mathrm{rinsing}}$ in bar')
plot_and_fill_area(ax1, 40, 70, T_fluid, 40, model_fluid_temperature, 'Fluid temperature', r'$\mathit{T}_{\mathrm{fluid}}$ in °C')
plot_and_fill_area(ax4, 45, 115, T_drying, 45, model_drying_air_temperature, 'Drying air temperature', r'$\mathit{T}_{\mathrm{drying}}$ in °C')

# Adjust layout
plt.tight_layout()
plt.show()

### Read current paramters, load models and visualize optimization potential (Black and white version for paper)

In [ ]:
%run KnowledgeBase.ipynb
%store -r server_url
%store -r node_ids
from matplotlib import rcParams

online_mode = False  # Set to True for online mode, False for offline mode

# Set font properties
rcParams['font.family'] = 'Palatino Linotype'
rcParams['font.size'] = 9

def get_value_within_range(prompt, min_value, max_value):
    """
    Function for manual entry of current parameter settings. 
    The min_value and max_value are technological boundary values defined in the knowledge base.

    Parameters:
    - promt (string): message 
    - min_value (float): Minimum value of paramter range.
    - max_value (float): Maximum value of paramter range.
    """
    
    while True:
        try:
            value = float(input(prompt))
            if min_value <= value <= max_value:
                return value
            else:
                print("\033[1;31mWarning: Value must be between {} and {}.\033[0m".format(min_value, max_value))
        except ValueError:
            print("\033[1;31mInvalid input. Please enter a number.\033[0m")

# Online mode
if online_mode:
    # Online mode
    try:
        
        # Creates a list of the node IDS extracted from the node_ids_dict from the knowledge base
        node_ids_list = list(node_ids_dict.values())
        
        # Get current parameters for the given node_ids_list via OPC UA
        result = read_opcua_values(server_url, node_ids_list)
        
        #  Dictionary, where the keys are the variable names (e.g. T_cleaning) from node_ids_dict, and the values are the current paramters
        result_dict = {key: value for key, value in zip(node_ids_dict.keys(), [x[1] for x in result])}
        
        # Extract individual variables
        T_fluid = result_dict['T_cleaning']
        p_cleaning = result_dict['p_cleaning']
        p_rinsing = result_dict['p_rinsing']
        T_drying = result_dict['T_drying']
        n_drying = result_dict['n_drying']
    
        print("T_fluid:", T_cleaning)
        print("p_cleaning:", p_cleaning)
        print("p_rinsing:", p_rinsing)
        print("T_drying:", T_drying)
        print("n_drying:", n_drying)
        
    except Exception as e:
        print("Error occurred while fetching data:", e)
        online_mode = False  # Switch to offline mode if an error occurs
        
# Offline mode  
if not online_mode:
        
    # Offline mode  
    try:
        n_drying = get_value_within_range("Enter current value for n_drying in rpm: ", 860, 3600)
        p_cleaning = get_value_within_range("Enter current value for p_cleaning in bar: ", 0.05, 2.3)
        p_rinsing = get_value_within_range("Enter current value for p_rinsing in bar: ", 0.05, 2.3)
        T_drying = get_value_within_range("Enter current value for T_drying in °C: ", 45, 115)
        T_fluid = get_value_within_range("Enter current value for T_fluid in °C: ", 40, 70)

    except Exception as e:
        print("Error occurred in offline mode:", e)
        

# Load the PMML file
current_path = os.getcwd()
model_drying_fan_speed_file_path = os.path.join(current_path +'/polynomial_regression_models/model_drying_fan_speed.pmml')
model_cleaning_pump_pressure_file_path = os.path.join(current_path +'/polynomial_regression_models/model_cleaning_pump_pressure.pmml')
model_rinsing_pump_pressure_file_path = os.path.join(current_path +'/polynomial_regression_models/model_rinsing_pump_pressure.pmml')
model_fluid_temperature_file_path = os.path.join(current_path +'/polynomial_regression_models/model_fluid_temperature.pmml')
model_drying_air_temperature_file_path = os.path.join(current_path +'/polynomial_regression_models/model_drying_air_temperature.pmml')


model_drying_fan_speed = Model.fromFile(model_drying_fan_speed_file_path)
model_cleaning_pump_pressure = Model.fromFile(model_cleaning_pump_pressure_file_path)
model_rinsing_pump_pressure = Model.fromFile(model_rinsing_pump_pressure_file_path)
model_fluid_temperature =  Model.fromFile(model_fluid_temperature_file_path)
model_drying_air_temperature = Model.fromFile(model_drying_air_temperature_file_path)

# Visulization of optimization potential

def plot_and_fill_area(ax, min_value, max_value, current_value, optimum_value, regression_model, plot_title, x_label):
    """
    Plot the regression model with the predicted value and fill the area
    between the operating point and the optimal operating point.

    Parameters:
    - ax: Axes object to plot on.
    - min_value (float): Minimum value of paramter range.
    - max_value (float): Maximum value of paramter range.
    - current_value (float): The current operating point.
    - optimum_value (float): The optimal operating point.
    - regression_model: The trained PMML model for predicting the active power.
    - plot_title(string): Title for plot.
    - x_label (string): Label for x-axis.
    """
    # Use the PMML model for prediction
    P = regression_model.predict([current_value])
    P_opt = regression_model.predict([optimum_value])

    # Plot the regression model with the predicted value
    x_values = np.linspace(min_value, max_value, 100).reshape(-1, 1)
    y_values = regression_model.predict(x_values)

    ax.plot(x_values, y_values, color="black", label='Regression Model',zorder=2)
    ax.scatter(current_value, P[0], color="black", alpha=0.5, zorder=3, label="Current operating point")
    ax.scatter(optimum_value, P_opt[0], color="black", marker='x', zorder=3, label="Optimum operating point")

    # Fill area under the curve between operating point and optimal operating point
    y_values = [item for sublist in y_values for item in sublist]
    ax.fill_between(x_values.ravel(), y_values, where=(x_values.ravel() >= optimum_value) & (x_values.ravel() <= current_value),  color='lightgrey',  zorder=1,  label='Energy saving potential')
    ax.set_xlabel(x_label)
    ax.set_ylabel(r'$\overline{P}$ in W')
    #ax.legend()
    #ax.set_title(plot_title)
    ax.grid(True)

    return P, P_opt


# Create figure and specify grid dimensions
fig = plt.figure(figsize=(8, 6))
fig.tight_layout(pad=5.0)


# Specify subplot locations
ax1 = fig.add_subplot(gs[0, 0])  # First row, first column
ax2 = fig.add_subplot(gs[0, 1])  # First row, second column
ax3 = fig.add_subplot(gs[0, 2])  # First row, third column
ax4 = fig.add_subplot(gs[1, 0])  # Second row, span first and second columns
ax5 = fig.add_subplot(gs[1, 1])  # Second row, third column

# Call the function with your data for each subplot
plot_and_fill_area(ax5, 860, 3600, n_drying, 860, model_drying_fan_speed, 'Drying fan speed', r'$\mathit{n}_{\mathrm{drying}}$ in rpm')
plot_and_fill_area(ax2, 0.05, 2.3, p_cleaning, 0.05, model_cleaning_pump_pressure, 'Cleaning pump pressure', r'$\mathit{p}_{\mathrm{cleaning}}$ in bar')
plot_and_fill_area(ax3, 0.05, 2.3, p_rinsing, 0.05, model_rinsing_pump_pressure, 'Rinsing pump pressure', r'$\mathit{p}_{\mathrm{rinsing}}$ in bar')
plot_and_fill_area(ax1, 40, 70, T_fluid, 40, model_fluid_temperature, 'Fluid temperature', r'$\mathit{T}_{\mathrm{fluid}}$ in °C')
plot_and_fill_area(ax4, 45, 115, T_drying, 45, model_drying_air_temperature, 'Drying air temperature', r'$\mathit{T}_{\mathrm{drying}}$ in °C')

# Adjust layout
# Create a global legend at the end
#Collect handles and labels for the legend
handles, labels = ax1.get_legend_handles_labels()

# Place the legend in the empty subplot area (lower right corner)
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.7, 0.3))

fig.tight_layout(pad=1.0)  # Adjust layout
plt.savefig('inlcude_path_where_to_save_figure', dpi=600)  # Optional: save with high resolution
plt.show()

In [ ]:
# Calculate EnPI

parameter_list = list(node_ids.keys())

# n_drying
P, P_opt = plot_and_fill_area(ax1, 860, 3600, n_drying, 860, model_drying_fan_speed, 'Drying fan speed', r'$\mathit{n}_{\mathrm{drying}}$ in rpm')
delta_P_n_drying = P[0] - P_opt[0]
O_n_drying = (n_drying-860)/(3600-860)

# p_cleaning
P, P_opt = plot_and_fill_area(ax2, 0.05, 2.3, p_cleaning, 0.05, model_cleaning_pump_pressure, 'Cleaning pump pressure', r'$\mathit{p}_{\mathrm{cleaning}}$ in bar')
delta_P_p_cleaning = P[0] - P_opt[0]
O_p_cleaning = (p_cleaning-0.05)/(2.3-0.05)

# p_rinsing
P, P_opt = plot_and_fill_area(ax3, 0.05, 2.3, p_rinsing, 0.05, model_rinsing_pump_pressure, 'Rinsing pump pressure', r'$\mathit{p}_{\mathrm{rinsing}}$ in bar')
delta_P_p_rinsing = P[0] - P_opt[0]
O_p_rinsing = (p_rinsing-0.05)/(2.3-0.05)

# T_fluid
P, P_opt = plot_and_fill_area(ax4, 40, 70, T_fluid, 40, model_fluid_temperature, 'Fluid temperature', r'$\mathit{T}_{\mathrm{fluid}}$ in °C')
delta_P_T_fluid = P[0] - P_opt[0]
O_T_fluid = (T_fluid-40)/(70-40)


# T_drying
P, P_opt = plot_and_fill_area(ax5, 45, 115, T_drying, 45, model_drying_air_temperature, 'Drying air temperature', r'$\mathit{T}_{\mathrm{drying}}$ in °C')
delta_P_T_drying = P[0] - P_opt[0]
O_T_drying = (T_drying-45)/(115-45)

delta_P_list = [delta_P_n_drying, delta_P_p_cleaning, delta_P_p_rinsing, delta_P_T_fluid, delta_P_T_drying]
O_list = [O_n_drying, O_p_cleaning, O_p_rinsing, O_T_fluid, O_T_drying]
parameter_list = ['n_drying', 'p_cleaning','p_rinsing', 'T_fluid', 'T_drying']

In [ ]:
def normalize(values):
    min_value = min(values)
    max_value = max(values)
    return [(value - min_value) / (max_value - min_value) for value in values]

delta_P_list_normalized = normalize(delta_P_list)
O_list_normalized = normalize(O_list)


In [ ]:
# Define the input variables for the fuzzy system

delta_P = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'Average active power')
O_cpv = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'Optimization potential of a CPV')

# Define the output variable
P_cpv = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'Priority number')

# Generate  membership functions for delta P
delta_P['low'] = fuzz.trapmf(delta_P.universe, [0, 0, 0.10, 0.15])
delta_P['medium'] = fuzz.trimf(delta_P.universe, [0.2, 0.5, 0.8])
delta_P['high'] = fuzz.trapmf(delta_P.universe, [0.7, 0.9, 1, 1])

# Generate  membership functions for optimization potential of a CPV
O_cpv['low'] = fuzz.trapmf(O_cpv.universe, [0, 0, 0.10, 0.15])
O_cpv['medium'] = fuzz.trimf(O_cpv.universe, [0.2, 0.5, 0.8])
O_cpv['high'] = fuzz.trapmf(O_cpv.universe, [0.7, 0.9, 1, 1])

# Generate  membership functions for priority number
P_cpv['low'] = fuzz.trapmf(P_cpv.universe, [0, 0, 0.20, 0.4])
P_cpv['medium'] = fuzz.trimf(P_cpv.universe, [0.2, 0.5, 0.8])
P_cpv['high'] = fuzz.trapmf(P_cpv.universe, [0.7, 0.9, 1, 1])

# Define fuzzy rule base
rule1 = ctrl.Rule(delta_P['high'] | O_cpv['low'], P_cpv['high'])
rule2 = ctrl.Rule(delta_P['medium'] | O_cpv['low'], P_cpv['medium'])
rule3 = ctrl.Rule(delta_P['low'] | O_cpv['low'], P_cpv['medium'])
rule4 = ctrl.Rule(delta_P['high'] | O_cpv['medium'], P_cpv['high'])
rule5 = ctrl.Rule(delta_P['medium'] | O_cpv['medium'], P_cpv['medium'])
rule6 = ctrl.Rule(delta_P['low'] | O_cpv['medium'], P_cpv['medium'])
rule7 = ctrl.Rule(delta_P['high'] | O_cpv['high'], P_cpv['medium'])
rule8 = ctrl.Rule(delta_P['medium'] | O_cpv['high'], P_cpv['low'])
rule9 = ctrl.Rule(delta_P['low'] | O_cpv['high'], P_cpv['low'])

# Create the control system
P_cpv_ctrl = ctrl.ControlSystem([rule1, rule2, rule3,rule4, rule5, rule6,rule7, rule8, rule9])
P_cpv_ctrl_simulation = ctrl.ControlSystemSimulation(P_cpv_ctrl)

# Compute the fuzzy inference for all inputs
for item in parameter_list:
    
    # Set inputs for simulation
    index = parameter_list.index(item)
    P_cpv_ctrl_simulation.input['Average active power'] = delta_P_list_normalized[index]
    P_cpv_ctrl_simulation.input['Optimization potential of a CPV'] = O_list_normalized[index]
    P_cpv_ctrl_simulation.compute()
    
    # Print the output
    print('Result for ', item , ' :\u0394P\u0305 = ',round(delta_P_list[index],2),' :\u0394p\u0305 = ',round(delta_P_list_normalized[index],2), ' W, O = ', round(O_list[index],2), 
          ' Priority number = ', round(P_cpv_ctrl_simulation.output['Priority number'],2))